Импорт библиотек

In [ ]:
import matplotlib.pyplot as plt
from random import randint
import csv

Глобальные переменные

In [ ]:
# Эти переменные объявлены глобально, потому что они задействуются почти во всех функциях CМО (система массового обслуживания)
# То есть так с ними просто удобней работать
# Остальные переменные задаются внутри функции
# Может это неочевидно и нехорошо так делать, но до лучшего я не додумался

l = 0                           # Размер очереди поступивших и необработанных заявок

t1, t2  = 25, 25                # Временные метки, необходимые для сбора статистики для построения графика
                                # (см. функции ApplicationReceiption и ApplicationProcessing)

s = 'free'                      # Статус обработчика

processed_apps_query = dict()   # Содержит временную метку в качестве ключа и кол-во обработанных заявок,
                                # обработанных в течении этой метки, в качестве значения

unprocessed_apps_query = dict() # Содержит временную метку в качестве ключа и кол-во  заявок,
                                # попавших в очередь в течение этой метки, в качестве значения
                                # Нужны для построения графика

Генератор очереди заявок

In [ ]:
# Симулирует случайное поступление заявок в реальном времени.
# Элемент очереди - это время поступление новой заявки
# Возвращает список с заявками

def QueueGenerator(a, b, tm):
    l = randint(1000, 2000)
    t = 0
    queue = []
    for _ in range(l):
        new_app = t + randint(a, b)
        if new_app < tm:
            queue.append(new_app)
            t = new_app
    return queue

Получение заявки

In [ ]:
# Принимает заявку
# Высчитывает временную метку когда заявка будет обработана с учетом уже имеющихся в очереди и
# возвращает эту временную метку

def ApplicationReceiption(time, t, c, d):
    global unprocessed_apps_query, s, l, t2
    event = max(time, t) + randint(c, d)
    if s == 'free':
        s = 'busy'
    else:
        l += 1
        while time > t2:
            t2 += 25
        try:
            unprocessed_apps_query[t2] += 1 # Собираем данные для графика
        except KeyError:
            unprocessed_apps_query[t2] = 1
    return event

Обработка заявки

In [ ]:
# Принимает заявку из очереди необработанных заявок
# Регулирует состояние обработчика

def ApplicationProcessing(time, tm):
    global t1, l, s, processed_apps_query, unprocessed_apps_query
    if time > tm:
        return
    if l > 0:
        l -= 1
        s = 'busy'
    else:
        s = 'free'
    while time > t1:
        t1 += 25
    try:
        processed_apps_query[t1] += 1   # Собираем данные для графика
    except KeyError:
        processed_apps_query[t1] = 1

Основная функция

In [ ]:
# Запускает работу СМО
# receiption_queue - очередь всех заявок
# processing_queue - очередь необработанных заявок

def MassServiceSystem(receiption_queue, processing_queue, a, b, c, d):

    t = 0                       # Время завершения обработки последней заявки в очереди поступивших заявок processing_queue
    tm = 500                    # Максимальое время работы системы
    time = 0                    # Текущий момет времени в системе

    processed_apps = 0          # Обработанные заявки
    unfinished_apps = 0         # Незавершенные заявки

    receiption_queue_is_not_empty = True # Метка, непуста ли очередь поступающих заявок

    while time < tm:        # Система работает до момента времени tm

        rec = -1            # Эти 2 переменные необходимыы для выбора, из какой очереди брать событие -
                            # из очереди поступающих (receiption_queue), или из очереди необработанных (processing_queue)
        proc = -1
        try:
            rec = receiption_queue[0]
        except IndexError:
            receiption_queue_is_not_empty = False
        try:
            proc = processing_queue[0]
        except IndexError:
            pass
        if receiption_queue_is_not_empty and len(processing_queue) and rec >= proc:
            e = processing_queue.pop(0)
            code = 2
        else:
            try:
                e = receiption_queue.pop(0)
                code = 1
            except IndexError:
                e = processing_queue.pop(0)
                code = 2
        time = e
        if code == 1:           # Код 1 - получаем новую заявку из receiption_queue
            event = ApplicationReceiption(time, t, c, d)
            t = event
            processing_queue.append(event)
            unfinished_apps += 1
        elif code == 2:         # Код 2 - обрабатываем заявку из processing_queue
            ApplicationProcessing(time, tm)
            processed_apps += 1
            unfinished_apps -= 1
    else:
        print('Время работы системы истекло. Завершение работы')
        print()
        return processed_apps, unfinished_apps  # По завершении работы возвращаем кол-во всех обработанных и необработанных заявок

Генератор метрик и и графиков

In [ ]:
def GraphicGenerator(a, b, c, d):
    global t1, t2, l, s, processed_apps_query, unprocessed_apps_query

    l = 0                       # Здесь обновляются значения глобальных переменных при новом вызове функции
    t1, t2  = 25, 25
    s = 'free'

    tm = 500

    processed_apps_query = dict()
    unprocessed_apps_query = dict()

    receiption_queue = QueueGenerator(a, b, tm) # Генерация receiption_queue
    processing_queue = []                       # processing_queue изначально пустой

    print(receiption_queue)

    result = MassServiceSystem(receiption_queue, processing_queue, a, b, c, d) # Запускаем СМО

    # Здесь делаем пост-обработку словарей для графиков
    # Заполняем нолями пропущенные временные метки

    if len(processed_apps_query) < 20:
        for i in range(1, 21):
            try:
                processed_apps_query[25 * i]
            except KeyError:
                processed_apps_query[25 * i] = 0

    if len(unprocessed_apps_query) < 20:
        for i in range(1, 21):
            try:
                unprocessed_apps_query[25 * i]
            except KeyError:
                unprocessed_apps_query[25 * i] = 0

    # Например, в processed_apps_query значение '50: 3' значит,
    # что в промежуток времени 25-50 было обработано 3 заявки
    print('Обработанные заявки по временным меткам')
    print(processed_apps_query)
    print()

    # в unprocessed_apps_query значение '50: 3' значит,
    # что в промежуток времени 25-50 было отправлено в очередь 3 заявки
    print('Заявки, находящиеся в очереди по временным меткам')
    print(unprocessed_apps_query)
    print()

    print(f'Всего поступивших заявок: {result[0] + result[1]}')
    print(f'Обработанные: {result[0]}')
    print(f'Необработанные: {result[1]}')
    print()

    # Составляем точки для графика из составленных словарей
    x_vals = [25 * i for i in range(1, 21)] # По оси Х берем ключи из словарей (временные метки)
    y_vals = []                             # По Y - отношение обработанных и необработанных заявок для соответствующих временных меток
    for i in range(1, 21):
        if processed_apps_query[25 * i] == 0 or (processed_apps_query[25 * i] == 0 and unprocessed_apps_query[25 * i] == 0): # Проводим обработку нулевых значений
            val = 0
        elif unprocessed_apps_query[25 * i] == 0:
            val = 3
        else:
            val = round(processed_apps_query[25 * i] / unprocessed_apps_query[25 * i], 2)
        y_vals.append(val)

    # Строим график

    plt.figure(figsize=(10, 7))
    plt.axes(ylim=(0, max(y_vals) + 2))
    plt.plot(x_vals, y_vals, 'o-r')
    plt.ylabel('Отношение обработанных заявок к необработанным')
    plt.xlabel('Временные метки')
    plt.show()

    # Визуально определяем начало стабильного промежутка - когда график перестает сильно колебаться
    begin = int(input('Введите начало стабильного промежутка: ')) // 25
    print()

    # Вычисляем среднее из значений по Y, попавших в стабильный промежуток
    mean_metric = sum([y_vals[i] for i in range(begin - 1, 20)]) / (20 - begin + 1)

    # Возвращаем набор из a, b, c, d и вычисленного среднего
    return [a, b, c, d, round(mean_metric, 2)]

Генератор набора метрик

In [ ]:
def MetricsGenerator():
    # Переменная, где будут храниться итоговые усредненные метрики
    metrics = []

    # Подбирает переменные
    # Учтите, что значения c и d должны быть больше или примерно равны соответственно a и b
    # Иначе график может отображаться некорректно
    for _ in range(5):
        print()
        print()
        print()
        print('==========')
        print()

        a = int(input('Введите значение a: '))
        b = int(input('Введите значение b: '))
        c = int(input('Введите значение c: '))
        d = int(input('Введите значение d: '))
        print()

        # Для каждого набора переменных 5 раз находит метрику и записывает в inter_metrics
        inter_metrics = []
        for i in range(5):
            inter_metrics.append(GraphicGenerator(a, b, c, d)[4])

        # Высчитывает финальную метрику как среднее из вычесленных
        final_metric = round(sum(inter_metrics) / 5, 2)

        # Записывает финальный набор из переменных и усредненной метрики
        metrics.append([a, b, c, d, final_metric])
    with open("metrics.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(metrics)
    # print(metrics)